## setup

In [25]:
from openai import OpenAI, AsyncOpenAI
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, AutoModelForSequenceClassification, DataCollatorForLanguageModeling
from huggingface_hub import login
from dotenv import load_dotenv
from datasets import load_dataset
import torch as t
import asyncio
import os
from peft import LoraConfig, TaskType, get_peft_model
import evaluate
import numpy as np
import bitsandbytes
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML
import wandb
import einops
from peft import LoraConfig, get_peft_model, TaskType
from tqdm import tqdm

In [3]:
load_dotenv()
login(token = os.environ['HF_TOKEN'])

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


## FT Dataset

### Load Dataset

In [ ]:
dataset_java = load_dataset('iNeil77/CodeNet', 'Java', split='train')
dataset_java = dataset_java.select_columns(['p_id', 'language', 'status', 'code'])
dataset_java = dataset_java.filter(lambda x: x['status']=='Accepted')
# dataset_java

In [ ]:
# dataset_js = load_dataset('iNeil77/CodeNet', 'JavaScript', split='train')
# dataset_js = dataset_js.select_columns(['p_id', 'language', 'status', 'code'])
# dataset_js = dataset_js.filter(lambda x: x['status']=='Accepted')
# # dataset_js

In [ ]:
# dataset_cpp = load_dataset('iNeil77/CodeNet', 'C++', split='train')
# dataset_cpp = dataset_cpp.select_columns(['p_id', 'language', 'status', 'code'])
# dataset_cpp = dataset_cpp.filter(lambda x: x['status']=='Accepted')
# # dataset_cpp

Resolving data files:   0%|          | 0/24 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

In [ ]:
# dataset_go = load_dataset('iNeil77/CodeNet', 'Go', split='train')
# dataset_go = dataset_go.select_columns(['p_id', 'language', 'status', 'code'])
# dataset_go = dataset_go.filter(lambda x: x['status']=='Accepted')
# # dataset_go

In [ ]:
# dataset_python = load_dataset('iNeil77/CodeNet', 'Python', split='train')
# dataset_python = dataset_python.select_columns(['p_id', 'language', 'status', 'code'])
# dataset_python = dataset_python.filter(lambda x: x['status']=='Accepted')
# # dataset_python

In [ ]:
# for i in range(10):
#     idx = random.randint(0, dataset_cpp.num_rows)
#     code = dataset_cpp[idx]['code']
#     print(code)
#     print('--x---x---x--\n')

#include <iostream>
#include<bits/stdc++.h>
using namespace std;
typedef long long ll;
ll dp[51],n,a,b,L,ok;
vector<ll> v[31];
int main(void){
    while(1){
    cin>>n;
    //cout<<n<<endl;
    if(n==0)return 0;
    ok=1;
    for(int i=0;i<31;i++)v[i]={};
    for(int i=0;i<n;i++){
        dp[i]=(1LL<<i);
    }
    for(int i=0;i<n;i++){
        cin>>a;
        for(int j=0;j<a;j++){
            cin>>b;
            v[b-1].push_back(i);
        }
    }
    for(int i=0;i<31;i++){
        L=0;
        for(auto x:v[i]){
            L|=dp[x];
        }
        if(L==(1LL<<n)-1){
            cout<<i+1<<endl;
            ok=0;
            break;
        }
        for(auto x:v[i]){
            dp[x]=L;
        }
    }
    if(ok)cout<<-1<<endl;
    }
}


--x---x---x--

#include<ctime>
#include<cmath>
#include<cstdio>
#include<cstring>
#include<iostream>
#include<algorithm>
#include<queue>
#include<vector>
#define p 1000000007
#define inv2 500000004
#define file(x)freopen(x".in","r",stdin);freopen(

### Fine-tune

In [28]:
model_name = 'meta-llama/Llama-3.1-8B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(model_name, pad_side="left")
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", device_map="auto")
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [29]:
def tokenize(record):
    code = record['code']
    tokens = tokenizer(
        code, 
        truncation=True,
        max_length=2048,
    )
    return tokens

tokenized_dataset_java = dataset_java.map(tokenize, batched=True, num_proc=16)

Map (num_proc=16):   0%|          | 0/348362 [00:00<?, ? examples/s]

In [30]:
tokenized_dataset_java

Dataset({
    features: ['p_id', 'language', 'status', 'code', 'input_ids', 'attention_mask'],
    num_rows: 348362
})

In [31]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # Adjust for Llama 3.1 if needed
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [32]:
training_args = TrainingArguments(
    output_dir="./llama3-java-finetune",
    # per_device_train_batch_size=2,  # Adjust based on GPU memory
    gradient_accumulation_steps=8,
    num_train_epochs=1,  
    learning_rate=2e-4,
    fp16=True,
    save_steps=1000,
    logging_steps=100,
    report_to="wandb"
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # For causal LM
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_java,
    tokenizer=tokenizer,
    data_collator=data_collator
)

/tmp/ipykernel_7370/2101947803.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [33]:
trainer.train()


OutOfMemoryError: CUDA out of memory. Tried to allocate 132.00 MiB. GPU 0 has a total capacity of 47.53 GiB of which 52.25 MiB is free. Process 3286969 has 47.47 GiB memory in use. Of the allocated memory 46.86 GiB is allocated by PyTorch, and 308.60 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

## Un-Focus

### HuggingFace Finetuning 

In [ ]:
dataset = load_dataset("karpathy/tiny_shakespeare")
print(dataset)
# print('hlelow')

In [ ]:
model_name = 'meta-llama/Llama-3.1-8B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_size='left')
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,  # Optional: saves memory
    device_map="auto",
    # torch_dtype="auto"
)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
questions = [
    'What is 5+5?', 
    'Tell me a poem.',
    'Tell me a story.',
]
message_template = [
    {"role": "system", "content": "You are a friendly, helpful chatbot."},
    {"role": "user", "content": ''},
]

def qualitative_eval():
    out = []
    for question in questions:
        message = message_template
        message[1]['content'] = question

        model_inputs = tokenizer.apply_chat_template(message, add_generation_prompt=True, return_tensors='pt').to('cuda')
        generated_ids = model.generate(model_inputs, do_sample=True, max_new_tokens=50)
        text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0].split('assistant')[-1]

        out.append(text)
    
    return out

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    num_proc=4,
    remove_columns=["text"]
)

In [ ]:
block_size = 256

def group_texts(examples):
    concatenated = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated["input_ids"])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_dataset = tokenized_dataset.map(
    group_texts,
    batched=True,
    num_proc=4
)

In [ ]:
training_args = TrainingArguments(
    output_dir="./llama3-shakespeare",
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    learning_rate=2e-4,
    bf16=True,
    logging_steps=10,
    save_steps=100,
    save_total_limit=2,
)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["validation"],
    tokenizer=tokenizer
)

trainer.train()

In [ ]:
qualitative_eval()

In [ ]:
trainer.save_model("./llama3-shakespeare")
tokenizer.save_pretrained("./llama3-shakespeare")

### HuggingFace Training (0/2)

In [ ]:
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')
datasets['train']['text']

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"]).to('cuda')

model_checkpoint = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

In [ ]:
block_size = tokenizer.model_max_length

first = True
my_result = ...
counter = 0

def group_texts(examples):
    global my_result, counter, first

    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size

    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()

    counter += 1
    if first:
        my_result = result
        first = False

    return result

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    # num_proc=4,
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_checkpoint, device_map='auto', torch_dtype='auto')

model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    eval_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

In [ ]:
trainer.train()

### HuggingFace Training (2/2)

In [ ]:
model_name = 'meta-llama/Llama-3.1-8B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left')
model = AutoModelForCausalLM.from_pretrained(model_name, device_map='auto', torch_dtype='auto')
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
questions = [
    'What is 5+5?', 
    'Tell me a poem.',
    'Tell me a story.',
]
message_template = [
    {"role": "system", "content": "You are a friendly, helpful chatbot."},
    {"role": "user", "content": ''},
]

def qualitative_eval():
    out = []
    for question in questions:
        message = message_template
        message[1]['content'] = question

        model_inputs = tokenizer.apply_chat_template(message, add_generation_prompt=True, return_tensors='pt').to('cuda')
        generated_ids = model.generate(model_inputs, do_sample=True, max_new_tokens=50)
        text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0].split('assistant')[-1]

        out.append(text)
    
    return out

out = qualitative_eval()
# for sentence in out:
#     print(sentence)

In [ ]:
def tokenize(text):
    return tokenizer(text['text'])

dataset = load_dataset('karpathy/tiny_shakespeare', trust_remote_code=True)
tokenized_dataset = dataset.map(tokenize, batched=True, remove_columns='text')

In [ ]:
block_size = 1028

def group_text(input):
    length = len(input['input_ids'])
    n_batches = length // block_size

    result = dict()

    for k, t in input.items():
        t = np.array(t[:n_batches * block_size])
        t = einops.rearrange(t, '(bat block) -> bat block', bat=n_batches)
        t = t.tolist()

        result[k] = t

    result['labels'] = result['input_ids'].copy()
    return result

lm_dataset = tokenized_dataset.map(group_text)

In [ ]:
training_args = TrainingArguments(
    f"finetuned-tiny-shakespeare",
    eval_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["validation"],
)

In [ ]:
trainer.train()

### HuggingFace Training (1/2)

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-3.1-8B-Instruct', padding_side='left')
# model = AutoModelForCausalLM.from_pretrained('meta-llama/Llama-3.1-8B-Instruct', device_map='auto', torch_dtype='auto')
# tokenizer.pad_token = tokenizer.eos_token

In [ ]:
ft_dataset = load_dataset('yelp_review_full')
ft_tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-cased')

def tokenize(examples):
    return ft_tokenizer(examples["text"], padding="max_length", truncation=True).to('cuda')
dataset = ft_dataset.map(tokenize, batched=True)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5)


In [ ]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(
    output_dir="yelp_review_classifier",
    eval_strategy="epoch",
)

In [ ]:
small_train = dataset["train"].shuffle(seed=42).select(range(1000))
small_eval = dataset["test"].shuffle(seed=42).select(range(1000))

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train,
    eval_dataset=small_eval,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

### HuggingFace Practice  
*getting comfortable with HF*

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-3.1-8B-Instruct', padding_side="left")
model = AutoModelForCausalLM.from_pretrained(
        'meta-llama/Llama-3.1-8B-Instruct', 
        device_map = 'auto',
        torch_dtype = 'auto',
)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
sentences = [
    'Jack n jill went up the hill to smoke a lot of pot',
    'Beautiful world, where are you?',
    'In the beginning, the universe was created. This was widely regarded as a bad move.',
]

out = tokenizer(sentences, padding=True, return_tensors='pt')['input_ids']


out2 = tokenizer.decode(out[0])

out3 = tokenizer.batch_decode(out, skip_special_tokens=True)

print(out2 + '\n')
print(out)

out3

In [ ]:
test_sentence = "What is 5+5? Reply with a single word."
model_inputs = tokenizer(test_sentence, return_tensors='pt').to('cuda')

generated = model.generate(**model_inputs, max_new_tokens=100, temperature=0.3, repetition_penalty= 2.0)
out = tokenizer.batch_decode(generated)

In [ ]:
print(out[0])

In [ ]:
model.generation_config

In [ ]:
messages = [
    {
        "role": "system",
        "content": "You are a friendly, helpful chatbot.",
    },
    {"role": "user", "content": "What is 5+5? Answer with a single word."},
]

model_inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to("cuda")
generated_ids = model.generate(model_inputs, do_sample=True, max_new_tokens=50)
print(tokenizer.batch_decode(generated_ids)[0])



### LoRA

In [ ]:
model

In [ ]:
# create LoRA configuration object
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, # type of task to train on
    inference_mode=False, # set to False for training
    r=8, 
    lora_alpha=32, 
    lora_dropout=0.1,
)

In [ ]:
model.add_adapter(lora_config, adapter_name="lora_1")


In [ ]:
model

### Async Test

In [ ]:
async def task1():
    print("Task 1: Start")
    await asyncio.sleep(2)
    print("Task 1: End")

async def task2():
    print("Task 2: Start")
    await asyncio.sleep(1)
    print("Task 2: End")

# async def main():
await asyncio.gather(task1(), task2())

# await main()

In [ ]:
async def chat_async():
    prompt = 'hello world'
    client = AsyncOpenAI()

    response = await client.responses.create(
        model = 'gpt-4.1-mini',
        input = prompt,
    )

    text_out = response.output[-1].content[0].text
    return text_out

def chat_reg():
    prompt = 'hello world'
    client = OpenAI()

    response = client.responses.create(
        model = 'gpt-4.1-mini',
        input = prompt,
    )

    text_out = response.output[-1].content[0].text
    return text_out

In [ ]:
await chat_async()